In [1]:
import pandas as pd
from sqlalchemy import create_engine

In [2]:
path = "Resources/Measurement_summary.csv"
seoul_air_df = pd.read_csv(path)

In [3]:
# Create a filtered dataframe from specific columns
seoul_cols = ["Measurement date", "Station code", "SO2", "NO2", "O3", "CO", "PM10", "PM2.5"]
seoul_transformed= seoul_air_df[seoul_cols].copy()

# Rename the column headers
seoul_transformed = seoul_transformed.rename(columns={"Measurement date": "measurement_date",
                                                          "Station code": "station_code",
                                                          "SO2": "sulfer_dioxide",
                                                           "NO2":"nitrous_dioxide",
                                                        "O3": "ozone",
                                                        "CO": "carbon_monoxide",
                                                        "PM10":"particulate_matter_10000nm",
                                                        "PM2.5":"particulate_matter_2500nm"})

In [4]:
seoul_transformed.insert(0, 'id', range(1, 1+ len(seoul_transformed)))

In [5]:
seoul_transformed.head()

,id,measurement_date,station_code,sulfer_dioxide,nitrous_dioxide,ozone,carbon_monoxide,particulate_matter_10000nm,particulate_matter_2500nm
0,1,2017-01-01 00:00,101,0.004,0.059,0.002,1.2,73.0,57.0
1,2,2017-01-01 01:00,101,0.004,0.058,0.002,1.2,71.0,59.0
2,3,2017-01-01 02:00,101,0.004,0.056,0.002,1.2,70.0,59.0
3,4,2017-01-01 03:00,101,0.004,0.056,0.002,1.2,70.0,58.0
4,5,2017-01-01 04:00,101,0.003,0.051,0.002,1.2,69.0,61.0


In [6]:
path_2 = "Resources/air-quality-monitoring-2014-1.csv"
uk_air_df = pd.read_csv(path_2)

In [7]:
#Need to change times that begin with "24" to "0" in order to utilize pd.DateTime
UK_narrow_1=uk_air_df.replace({'End Time': '24:00:00'},'00:00:00')

In [8]:
#combine date and time columns, convert to DateTime
combined=UK_narrow_1['End Date'] + ' ' + UK_narrow_1['End Time']
combined_dt=pd.to_datetime(combined, yearfirst=True)
UK_narrow_1['measurement_timestamp']=combined_dt

In [9]:
# Create a filtered dataframe from specific columns
UK_cols = ["Air Quality Monitoring Station", "measurement_timestamp", "PM10", "Units", "NO2"]
UK_narrow= UK_narrow_1[UK_cols].copy()

In [10]:
# Rename the column headers
UK_transformed = UK_narrow.rename(columns={"NO2":"nitrous_dioxide",
                                                        "Air Quality Monitoring Station": "station_name",
                                                        "PM10":"particulate_matter_10000nm",
                                                        "NO2":"nitrous_dioxide_ugm_3",
                                                        "Units" : "units"})

In [11]:
UK_transformed.insert(0, 'id', range(1, 1+ len(UK_transformed)))

In [12]:
UK_transformed.head()

,id,station_name,measurement_timestamp,particulate_matter_10000nm,units,nitrous_dioxide_ugm_3
0,1,Tally Ho AQMS,2014-01-01 01:00:00,23.0,ugm-3 (INDIC.GRAV),31.0
1,2,Tally Ho AQMS,2014-01-01 02:00:00,16.0,ugm-3 (INDIC.GRAV),34.0
2,3,Tally Ho AQMS,2014-01-01 03:00:00,12.0,ugm-3 (INDIC.GRAV),36.0
3,4,Tally Ho AQMS,2014-01-01 04:00:00,16.0,ugm-3 (INDIC.GRAV),36.0
4,5,Tally Ho AQMS,2014-01-01 05:00:00,18.0,ugm-3 (INDIC.GRAV),38.0


In [21]:
connection_string = "postgres:Ceilidh91!@localhost:5432/ETL_project"
engine = create_engine(f'postgresql://{connection_string}')

In [22]:
# Confirm tables
engine.table_names()

['seoul', 'uk']

In [23]:
UK_transformed.to_sql(name='uk', con=engine, if_exists='append', index=False)

In [24]:
seoul_transformed.to_sql(name='seoul', con=engine, if_exists='append', index=False)

In [25]:
pd.read_sql_query('select * from uk', con=engine).head()

,id,station_name,measurement_timestamp,particulate_matter_10000nm,units,nitrous_dioxide_ugm_3
0,1,Tally Ho AQMS,2014-01-01 01:00:00,23.0,ugm-3 (INDIC.GRAV),31.0
1,2,Tally Ho AQMS,2014-01-01 02:00:00,16.0,ugm-3 (INDIC.GRAV),34.0
2,3,Tally Ho AQMS,2014-01-01 03:00:00,12.0,ugm-3 (INDIC.GRAV),36.0
3,4,Tally Ho AQMS,2014-01-01 04:00:00,16.0,ugm-3 (INDIC.GRAV),36.0
4,5,Tally Ho AQMS,2014-01-01 05:00:00,18.0,ugm-3 (INDIC.GRAV),38.0


In [26]:
pd.read_sql_query('select * from seoul', con=engine).head()

,id,measurement_date,station_code,sulfer_dioxide,nitrous_dioxide,ozone,carbon_monoxide,particulate_matter_10000nm,particulate_matter_2500nm
0,1,2017-01-01 00:00:00,101,0.004,0.059,0.002,1.2,73.0,57.0
1,2,2017-01-01 01:00:00,101,0.004,0.058,0.002,1.2,71.0,59.0
2,3,2017-01-01 02:00:00,101,0.004,0.056,0.002,1.2,70.0,59.0
3,4,2017-01-01 03:00:00,101,0.004,0.056,0.002,1.2,70.0,58.0
4,5,2017-01-01 04:00:00,101,0.003,0.051,0.002,1.2,69.0,61.0
